In [1]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img



# Data paths (adjust these to the actual image paths)
genuine_image_dir = '/Users/logpoint/Documents/SoftwareProjects/Thesis/CASIA22/Au'
forged_image_dir = '/Users/logpoint/Documents/SoftwareProjects/Thesis/CASIA22/Tp'


# Parameters
image_size = 160

In [2]:

# Function to preprocess images (resize and normalize)
def preprocess_image(image_path):
    image = load_img(image_path, target_size=(image_size, image_size))
    image = img_to_array(image)
    image = image / 255.0 # Normalize to [0, 1]
    return image

# Error Level Analysis (ELA) function
def ela_image(image_path, quality=90):
    original_image = load_img(image_path, target_size=(image_size, image_size))
    original_image = img_to_array(original_image)

    # Save the image at a lower quality
    temp_image_path = "temp_ela_image.jpg"
    cv2.imwrite(temp_image_path, cv2.imread(image_path), [int(cv2.IMWRITE_JPEG_QUALITY), quality])

    # Re-load the compressed image
    compressed_image = load_img(temp_image_path, target_size=(image_size, image_size))
    compressed_image = img_to_array(compressed_image)

    # Calculate the difference (Error Level)
    ela_diff = np.abs(original_image - compressed_image)

    # Normalize the difference image for better visibility
    max_ela = np.max(ela_diff)
    ela_diff = (ela_diff / max_ela) * 255.0

    return ela_diff

# Load and preprocess dataset
def load_dataset():
    images = []
    labels = []

    # Load genuine images
    for img_file in os.listdir(genuine_image_dir):
        try:
            img_path = os.path.join(genuine_image_dir, img_file)
            ela_img = ela_image(img_path)
            images.append(ela_img)
            labels.append(0) # Label 0 for genuine
        except:
            continue

    # Load forged images
    for img_file in os.listdir(forged_image_dir):
        try:
            img_path = os.path.join(forged_image_dir, img_file)
            ela_img = ela_image(img_path)
            images.append(ela_img)
            labels.append(1) # Label 1 for forged
        except:
            continue

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Build CNN model
def build_model():
    model = Sequential()

    # First Conv layer with 32 filters and 5x5 kernel size
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(image_size, image_size, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Conv layer with 32 filters and 5x5 kernel size
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Dropout to prevent overfitting
    model.add(Dropout(0.5))

    # Flatten layer to convert feature maps to a 1D vector
    model.add(Flatten())

    # Fully connected layer (Dense) for pattern recognition
    model.add(Dense(128, activation='relu'))

    # Dropout in dense layer
    model.add(Dropout(0.5))

    # Output layer with softmax for binary classification
    model.add(Dense(2, activation='softmax')) # 2 classes: Genuine and Forged

    return model


In [3]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


learning_rate = 0.0001
batch_size = 32
epochs = 10 # Limiting epochs to avoid overfitting


# Load dataset
images, labels = load_dataset()

# Split data into training and validation sets (80% train, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

# Build model
model = build_model()

# Compile model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs)

# Evaluate model
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

/Users/logpoint/Documents/SoftwareProjects/Thesis/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 184s 580ms/step - accuracy: 0.6122 - loss: 3.1058 - val_accuracy: 0.7622 - val_loss: 0.4467
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 180s 571ms/step - accuracy: 0.7700 - loss: 0.4678 - val_accuracy: 0.7780 - val_loss: 0.4254
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 179s 567ms/step - accuracy: 0.7864 - loss: 0.4458 - val_accuracy: 0.8367 - val_loss: 0.3868
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 178s 563ms/step - accuracy: 0.8312 - loss: 0.3848 - val_accuracy: 0.8450 - val_loss: 0.3532
Epoch 5/10
 18/316 ━━━━━━━━━━━━━━━━━━━━ 2:58 599ms/step - accuracy: 0.8639 - loss: 0.3168

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()